In [177]:
import qnexus as qnx
import pytket as pk
import numpy as np
import scipy as sc

In [178]:
project = qnx.projects.get_or_create(name="U1_TEBD")
qnx.context.set_active_project(project)
config = qnx.QuantinuumConfig(device_name="H2-Emulator")

In [179]:
import operators.plaqOps as test

In [ ]:
g2 = 1.0

In [181]:
oL, Lsq, ImTrP = test._getPlaqStateOps(1, g2)

In [182]:
oL, Lsq, ImTrP = np.pad(oL, (0,1)), np.pad(Lsq, (0,1)), np.pad(ImTrP, (0,1))

In [183]:
operator_matrix1 = sc.linalg.expm(-1j*np.matrix(g2/2 * 4* Lsq + 1/2/g2*ImTrP))

In [184]:
from pytket.circuit import Circuit
from pytket.utils import QubitPauliOperator
from pytket.passes import DecomposeBoxes
import numpy as np

from pytket.circuit import Unitary2qBox

circuit2 = Circuit(2)

# Create a Unitary2qBox from the matrix and add it to the circuit
unitary_box = Unitary2qBox(operator_matrix1)
circuit2.add_unitary2qbox(unitary_box, 0, 1)

# Decompose the box into basic gates
DecomposeBoxes().apply(circuit2)


True

In [185]:
render_circuit_jupyter(circuit2)

In [186]:
operator_matrix2 = sc.linalg.expm(-1j*np.matrix(g2/2 * np.kron(oL, oL)))

In [187]:
from qiskit import QuantumCircuit, transpile
from pytket.extensions.qiskit import qiskit_to_tk

# For 4-qubit gates, use qiskit to create the unitary and convert to pytket
print(f"Operator matrix2 shape: {operator_matrix2.shape}")

if operator_matrix2.shape == (16, 16):
    # Create a qiskit circuit with the unitary
    qc = QuantumCircuit(4)
    qc.unitary(operator_matrix2, [0, 1, 2, 3], label='U4')
    
    # Use qiskit's transpiler with optimization
    qc_decomposed = transpile(qc, basis_gates=['u3', 'cx'], optimization_level=3)
    
    # Convert to pytket circuit
    circuit3 = qiskit_to_tk(qc_decomposed)
    
    # Apply compatible optimization passes
    from pytket.passes import (
        DecomposeBoxes,
        FullPeepholeOptimise,
        CliffordSimp,
        RemoveRedundancies,
        SimplifyInitial
    )
    
    # Step-by-step optimization to avoid predicate conflicts
    DecomposeBoxes().apply(circuit3)
    CliffordSimp().apply(circuit3)  # Simplify Clifford gates first
    FullPeepholeOptimise().apply(circuit3)  # Aggressive optimization
    RemoveRedundancies().apply(circuit3)
    SimplifyInitial(allow_classical=False).apply(circuit3)  # Simplify at circuit boundaries
    
    # One more optimization pass
    FullPeepholeOptimise().apply(circuit3)
    
    print(f"Circuit synthesized and optimized successfully")
else:
    print(f"Warning: Expected 16x16 matrix, got {operator_matrix2.shape}")

Operator matrix2 shape: (16, 16)
Circuit synthesized and optimized successfully


In [188]:
render_circuit_jupyter(circuit3)

In [189]:
# Analyze circuit3 quality
print(f"Number of gates: {circuit3.n_gates}")
print(f"Depth: {circuit3.depth()}")
print(f"Two-qubit gate count: {circuit3.n_gates_of_type(pk.circuit.OpType.CX)}")

# Check unitary fidelity (verify decomposition is accurate)
original_unitary3 = np.array(operator_matrix2)
reconstructed_unitary3 = circuit3.get_unitary()

# Calculate process fidelity: F = |tr(U† V)|² / d²
d = original_unitary3.shape[0]
trace_product = np.trace(original_unitary3.conj().T @ reconstructed_unitary3)
fidelity3 = np.abs(trace_product)**2 / d**2

print(f"\nFidelity: {fidelity3:.10f}")
print(f"Infidelity: {1 - fidelity3:.2e}")

Number of gates: 255
Depth: 187
Two-qubit gate count: 95

Fidelity: 0.9999820926
Infidelity: 1.79e-05
